In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [37]:
# Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [38]:
# Delete unnecessary columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [39]:
# Label encodeing for Gender column
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data[['Gender']])
data['Gender'].value_counts()

d:\ML\ANN Classification\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Gender
1    5457
0    4543
Name: count, dtype: int64

In [40]:
# OneHot encoding for Geography column
from sklearn.preprocessing import OneHotEncoder
one_hot_encoding_geography = OneHotEncoder()
geography_encoded = one_hot_encoding_geography.fit_transform(data[['Geography']])
geography_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [41]:
geography_encoded.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [42]:
one_hot_encoding_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [43]:
geography_encoded_df = pd.DataFrame(geography_encoded.toarray(), columns=one_hot_encoding_geography.get_feature_names_out(['Geography']))
data = pd.concat([data, geography_encoded_df], axis=1)
data = data.drop(['Geography'], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [44]:
# Save the encoder and scaler

with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('one_hot_encoding_geography.pkl', 'wb') as f:
    pickle.dump(one_hot_encoding_geography, f)


In [45]:
# Divide data into training and testing

X = data.drop('Exited', axis=1)
y = data['Exited']

# train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [46]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [47]:
# save scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


# ANN Implementation

In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [49]:
# Build our ANN Model

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 connected with input layer 64 neurons
    Dense(32, activation='relu'), # HL2 connected with 32 neurons
    Dense(1, activation='sigmoid') # output layer
])

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


where params means addition of all weights and bias

In [51]:
# compile the model
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001) # optimizer is adam we can directly initialize it while compiling
loss = tensorflow.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])


In [52]:
# Setup the tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [53]:
# Setup EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [54]:
# Train the model

history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs=100,
    callbacks=[tensorboard_callback, early_stopping]
)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.4458 - accuracy: 0.8044 - val_loss: 0.3893 - val_accuracy: 0.8310
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3803 - accuracy: 0.8460 - val_loss: 0.3568 - val_accuracy: 0.8515
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3534 - accuracy: 0.8553 - val_loss: 0.3497 - val_accuracy: 0.8585
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3435 - accuracy: 0.8614 - val_loss: 0.3443 - val_accuracy: 0.8590
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3377 - accuracy: 0.8593 - val_loss: 0.3438 - val_accuracy: 0.8575
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3349 - accuracy: 0.8601 - val_loss: 0.3424 - val_accuracy: 0.8595
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3313 - accuracy: 0.8635 - val_loss: 0.3422 - val_accuracy: 0.8605

In [55]:
# Save the model
model.save("model.h5")


d:\ML\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [56]:
# Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [57]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 12136), started 6:47:34 ago. (Use '!kill 12136' to kill it.)